In [1]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'Times New Roman'

from scipy import stats

# Try to import Breusch-Pagan test
_has_bp = True
try:
    import statsmodels.api as sm
    from statsmodels.stats.diagnostic import het_breuschpagan
except Exception:
    _has_bp = False

In [3]:
# I/O
output_dir = r"E:\jupyternoteBookWorkPath\erp\house_eco_poi1"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "analysis_results.txt")


In [5]:
with open(output_file, 'w', encoding='utf-8') as f:
    # data
    try:
        df = pd.read_csv('shanghai_house_price_9_with_macro_3mavg.csv')
    except FileNotFoundError:
        f.write("Error: CSV file 'shanghai_house_price_9_with_macro_3mavg.csv' not found. Please ensure it is in the working directory.\n")
        raise

    physical_features = ['bedroom_num', 'livingroom_num', 'area', 'finish_level', 'age']
    neighborhood_features = [
        'floor_area_ratio','green_space_ratio',
        'dist_to_nearest_subway_station_m','dist_to_bank_m',
        'dist_to_primary_school_m','dist_to_middle_school_m',
        'dist_to_shopping_center_m','dist_to_top_tier_hospital_m',
        'dist_to_scenic_spot_m'
    ]
    macro_features = [
        'gdp_growth_yoy_3m_avg','gdp_growth_qoq_3m_avg',
        'cpi_yoy_3m_avg','cpi_mom_3m_avg',
        'ppi_yoy_3m_avg','ppi_mom_3m_avg'
    ]

    y = df['unit_price']

    # 4.1 EDA
    f.write("\n=== 4.1 Exploratory Data Analysis ===\n")
    f.write("Summary Statistics:\n")
    f.write(str(df[physical_features + neighborhood_features + macro_features + ['unit_price']].describe()))

    # Unit price histogram
    plt.figure(figsize=(10, 6))
    sns.histplot(df['unit_price'], bins=30)
    plt.title('Distribution of Unit Price')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'unit_price_histogram.png'), dpi=300)
    plt.close()
    f.write("\nHistogram of unit price saved as 'unit_price_histogram.png'")

    # Correlation heatmap
    plt.figure(figsize=(27, 21))
    corr = df[physical_features + neighborhood_features + macro_features + ['unit_price']].corr()
    sns.heatmap(corr, annot=False, cmap='coolwarm')
    plt.title('Correlation Heatmap', fontsize=28)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'correlation_heatmap.png'), dpi=300)
    plt.close()
    f.write("\nCorrelation heatmap saved as 'correlation_heatmap.png'")

    # 3.2 Model Configuration
    params = {
        'n_estimators': 100,
        'max_depth': 6,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
        'eval_metric': 'rmse'
    }
    f.write("\n\n=== 3.2 Model Configuration ===\n")
    for k, v in params.items():
        f.write(f"{k}: {v}\n")

    # Scenarios
    scenarios = {
        'Physical Only': physical_features,
        'Physical + Neighborhood': physical_features + neighborhood_features,
        'Physical + Neighborhood + Macro': physical_features + neighborhood_features + macro_features
    }

    results = {}

    # Train,Evaluate
    for scenario_name, features in scenarios.items():
        f.write(f"\n=== 4.2–4.6 Results for {scenario_name} ===\n")

        X = df[features]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        model = xgb.XGBRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=False
        )

        # 4.2 Training outcomes (learning curves)
        evals_result = model.evals_result()
        n_iter = len(evals_result['validation_0']['rmse'])
        final_train_rmse = evals_result['validation_0']['rmse'][-1]
        final_test_rmse = evals_result['validation_1']['rmse'][-1]
        f.write(f"Number of iterations: {n_iter}\n")
        f.write(f"Final training RMSE: {final_train_rmse:.2f}\n")
        f.write(f"Final test RMSE: {final_test_rmse:.2f}\n")

        plt.figure(figsize=(10, 6))
        plt.plot(evals_result['validation_0']['rmse'], label='Training RMSE')
        plt.plot(evals_result['validation_1']['rmse'], label='Test RMSE')
        plt.title(f'Learning Curves - {scenario_name}')
        plt.xlabel('Iteration'); plt.ylabel('RMSE'); plt.legend()
        fn_curve = f'learning_curve_{scenario_name.replace(" ", "_").lower()}.png'
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, fn_curve), dpi=300)
        plt.close()
        f.write(f"Learning curve saved as '{fn_curve}'\n")

        # 4.3 Prediction Results
        y_pred = model.predict(X_test)

        # 4.4 Error Metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        results[scenario_name] = {'RMSE': rmse, 'MAE': mae, 'R2': r2}

        f.write("\nEvaluation Metrics:\n")
        f.write(f"RMSE: {rmse:.2f}\nMAE: {mae:.2f}\nR²: {r2:.4f}\n")

        # 4.5 Residual Diagnostics (replaces feature importance/SHAP)
        residuals = y_test - y_pred
        res_mean = residuals.mean()
        res_std = residuals.std(ddof=1)
        res_skew = stats.skew(residuals, nan_policy='omit')
        res_kurt = stats.kurtosis(residuals, fisher=True, nan_policy='omit')

        # Shapiro–Wilk (subsample if very large)
        shapiro_sample = residuals
        if residuals.shape[0] > 5000:
            shapiro_sample = residuals.sample(5000, random_state=42)
        try:
            sh_w, sh_p = stats.shapiro(shapiro_sample)
        except Exception:
            sh_w, sh_p = np.nan, np.nan

        f.write("\nResidual summary:\n")
        f.write(f"Mean: {res_mean:.4f}, Std: {res_std:.4f}, Skew: {res_skew:.4f}, Kurtosis: {res_kurt:.4f}\n")
        f.write(f"Shapiro–Wilk W={sh_w:.4f}, p={sh_p:.4g} (normality test; larger p suggests closer to normal)\n")

        # optional Breusch–Pagan (requires statsmodels)
        if _has_bp:
            # OLS on fitted values to test heteroskedasticity in residuals
            exog = sm.add_constant(y_pred)  # simplest spec: variance ~ fitted
            bp_stat, bp_p, _, _ = het_breuschpagan(residuals, exog)
            f.write(f"Breusch–Pagan: LM={bp_stat:.3f}, p={bp_p:.4g} (heteroskedasticity test)\n")
        else:
            f.write("Breusch–Pagan test skipped (statsmodels not available).\n")

        # plot: residuals vs fitted
        plt.figure(figsize=(9, 6))
        plt.scatter(y_pred, residuals, s=10, alpha=0.6)
        plt.axhline(0, color='black', linewidth=1)
        plt.xlabel('Fitted (Predicted)'); plt.ylabel('Residuals')
        plt.title(f'Residuals vs Fitted – {scenario_name}')
        fn_rvf = f"residuals_vs_fitted_{scenario_name.replace(' ', '_').lower()}.png"
        plt.tight_layout(); plt.savefig(os.path.join(output_dir, fn_rvf), dpi=300); plt.close()
        f.write(f"Residuals vs Fitted plot saved as '{fn_rvf}'\n")

        # plot: histogram / density of residuals
        plt.figure(figsize=(9, 6))
        sns.histplot(residuals, bins=40, stat='density', alpha=0.8)
        # overlay normal PDF with same mean/std
        xs = np.linspace(residuals.min(), residuals.max(), 400)
        pdf = stats.norm.pdf(xs, loc=res_mean, scale=res_std if res_std > 0 else 1.0)
        plt.plot(xs, pdf, linewidth=2)
        plt.title(f'Residual Distribution – {scenario_name}')
        plt.xlabel('Residual'); plt.ylabel('Density')
        fn_hist = f"residual_hist_{scenario_name.replace(' ', '_').lower()}.png"
        plt.tight_layout(); plt.savefig(os.path.join(output_dir, fn_hist), dpi=300); plt.close()
        f.write(f"Residual histogram saved as '{fn_hist}'\n")

        # plot: Q–Q
        plt.figure(figsize=(9, 6))
        (osm, osr), (slope, intercept, r) = stats.probplot(residuals, dist='norm')
        plt.scatter(osm, osr, s=10, alpha=0.6)
        plt.plot(osm, slope*osm + intercept, linewidth=2)
        plt.title(f'Q–Q Plot of Residuals – {scenario_name}')
        plt.xlabel('Theoretical Quantiles'); plt.ylabel('Ordered Residuals')
        fn_qq = f"residual_qq_{scenario_name.replace(' ', '_').lower()}.png"
        plt.tight_layout(); plt.savefig(os.path.join(output_dir, fn_qq), dpi=300); plt.close()
        f.write(f"Residual Q–Q plot saved as '{fn_qq}'\n")

        # plot: standardized residuals (index order)
        std_res = (residuals - res_mean) / (res_std if res_std > 0 else 1.0)
        plt.figure(figsize=(9, 6))
        plt.plot(range(len(std_res)), std_res, marker='o', linestyle='none', markersize=3, alpha=0.7)
        plt.axhline(0, color='black', linewidth=1)
        plt.axhline(2, color='red', linestyle='--', linewidth=1)
        plt.axhline(-2, color='red', linestyle='--', linewidth=1)
        plt.title(f'Standardized Residuals – {scenario_name}')
        plt.xlabel('Observation Index'); plt.ylabel('Standardized Residual')
        fn_std = f"residual_standardized_{scenario_name.replace(' ', '_').lower()}.png"
        plt.tight_layout(); plt.savefig(os.path.join(output_dir, fn_std), dpi=300); plt.close()
        f.write(f"Standardized residuals plot saved as '{fn_std}'\n")

    # 4.6 ablation study (scenario comparison)
    f.write("\n=== 4.6 Ablation Study Results ===\n")
    f.write("Scenario\tRMSE\tMAE\tR²\n")
    for scenario_name, m in results.items():
        f.write(f"{scenario_name}\t{m['RMSE']:.2f}\t{m['MAE']:.2f}\t{m['R2']:.4f}\n")

print(f"Analysis complete. Results saved to {output_file}")

Analysis complete. Results saved to E:\jupyternoteBookWorkPath\erp\house_eco_poi1\analysis_results.txt
